# Google Cloud Machine Learning Engine

One of the main advantages of using the `Estimator` API is that it makes distributed training work essentially "out of the box". Now that we've gotten a model and a data pipeline to feed it established, let's turn to using Google Cloud Machine Learning Engine (CMLE) to distribute model training.

In addition to distributed training on the cloud, there are a few other advantages of CMLE:
1. It removes model training from consuming local computational resources, so you can not worry about chocking the system while training a model.
2. It has a nice model versioning system that allows you to swap out updated models with minimal disruption to your prediction pipeline.
3. It automatically scales to handle any volume of online predictions sent to the model.
4. It provides an excellent hyper-parameter tuning engine using Bayesian optimization (think suped up random search).

## 1. Preparing the data

In order for our model to train on the cloud, the training (and evaluation) data also needs to be on the cloud. CMLE requires these data to be stored in Google Cloud Storage (GCS). Cloud storage stores data in "buckets", which is basically a data lake (just a giant blob of data; no folders/file tree). You can organize the data in a bucket by just naming things as if there were folders. Google's cloud platform console will navigate as if there are actually folders if you name things this way.

There are three ways of interacting with GCP: command line utilities, the web console, and Python APIs. We'll just do things with command line here. Maybe in a follow up I'll cover the Python APIs.

The first thing to do is make sure you make sure your credentials are visible to the GCP APIs by running
```bash
gcloud auth list
```
and verifying that your account is set as the active user. If you don't see your account, run 
```bash
gcloud config set account <your account>
```
and follow the instructions as prompted.

To make a bucket, run
```bash
gsutil mb <name of the bucket>
```
You can tell a URI is a bucket because it will start with `gs://`. The naming convention for buckets is to use all lowercase letters and numbers, and dashes to separate words (not underscores). 

In [18]:
import datetime
import os

In [12]:
BUCKET_NAME = 'boston-ml'
os.environ['BUCKET_NAME'] = BUCKET_NAME

In [13]:
# %%bash
# gsutil mb $BUCKET_NAME

Now that we have a location for the training and evaluation data, we can copy it to our bucket using `gsutil cp`.

In [15]:
TRAIN_DATA_LOCAL = 'data/train'
EVAL_DATA_LOCAL = 'data/test'
os.environ['TRAIN_DATA_LOCAL'] = TRAIN_DATA_LOCAL
os.environ['EVAL_DATA_LOCAL'] = EVAL_DATA_LOCAL

In [16]:
# %%bash
# gsutil cp $TRAIN_DATA_LOCAL gs://$BUCKET_NAME/data/train/
# gsutil cp $EVAL_DATA_LOCAL gs://$BUCKET_NAME/data/test/

## 2. Preparing the trainer module

Cloud MLE needs a `.tar` of the Python package (or the scripts necessary to build the package) and a `config.yaml` file to train the model. It is easiest to supply the Python package as a bunch of scripts, including a `setup.py` script and letting CMLE do the packaging for you. In this approach, our model will need to be refactored into the structure:

```
cmle_trainer/
    trainer/
        __init__.py
        model.py
        task.py
    setup.py
    config.yaml
```

The `trainer` directory will be the Python package we make. It is packaged by the `setup.py` script. Both of these parts are the standard way Python packages are written: the package is a directory with all of the Python modules in it and an `__init__.py` file to let Python know that folder is a Python package; the `setup.py` script is a script that uses the `setuptools` package to install. The only thing special here is the `config.yaml` file which is used to configure how CMLE runs during training, and that the `trainer` package has two modules `model.py` and `task.py`. The `config.yaml` file is entirely optional, and we aren't going to use it right now.

The `task.py` module is where the `tf.estimator.train_and_evaluate` function is actually called, and is what CMLE runs while training the model. Any parameters of the script we want to be able to change from training run to training run (such as hyper-parameters for tuning or the path to the training/eval files) need to be able to be passed to `task.py` as command line arguments. To work with CMLE, we are required to have a `--job-dir` flag, which points to the directory in which the model binaries are stored (the `model_dir` of the `Estimator`).

The `model.py` module is where the actual model itself lives. This is basically the code from the previous notebook refactored a bit. The two biggest changes are:
1. Many parameters are now collected as attributes of a `flags` object. This lets us keep all parameters that we want to interact with via command line together.
2. The code is slightly more organized into functional blocks because programming.

**Note:** If you have any packages that are not available by default in the Cloud ML Engine runtime version you are using, they need to be specified in the `setup.py` script. There are some surprising omissions from the runtime default packages, such as the standard library package `pathlib`.

## 3. Submitting a training job

Prediction, training, and hyper-parameter are all done by submitting "jobs" to CMLE. Different job types have different requirements. To submit a training job, the command will be along the following lines:

In [19]:
coda = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
JOB_NAME = 'job_{0}'.format(coda)
MODEL_NAME = 'model_{0}'.format(coda)
os.environ['JOB_NAME'] = JOB_NAME
os.environ['MODEL_NAME'] = MODEL_NAME

In [20]:
# %%bash
# gcloud ml-engine jobs submit training $JOB_NAME \
#     --staging-bucket gs://$BUCKET_NAME/staging/ \
#     --job-dir gs://$BUCKET_NAME/model/$MODEL_NAME/ \
#     --package-path cmle_trainer/trainer \
#     --module-name trainer.task \
#     --region us-east1 \
#     --runtime-version 1.12 \
#     --python-version 3.5 \
#     --config cmle_trainer/config.yaml \
#     -- \
#     --train_data_dir gs://$BUCKET_NAME/data/train/ \
#     --eval_data_dir gs://$BUCKET_NAME/data/test/ \
#     --batch_size 32

There are a host of parameters to get through. First, notice there is a line that is just `--`. Everything above it is a flag for the `gcloud ml-engine jobs submit training` utility; everything below it is an argument that can be passed into the `task.py` module.

- The job name specifies 
- `staging-bucket` is a location in cloud storage. Cloud MLE makes a directory in the staging bucket with the name of the job in which the Python package is built using `setup.py`, and where the resulting `.tar.gz` file is located.
- `job-dir` is the directory where all of the model log files should be written. This flag behaves like one of the command line flags below the dividing `--` in that it gets passed to the `task.py` module. The estimator's `model_dir` needs to point to the same location as `job-dir`.
- `package-path` is the path to the `trainer` package on your local machine.
- `module-name` is the name of the module that runs the training. It is where the `tf.estimator.train_and_evaluate` function is actually called.
- `region` specifies the region the job should be computed in.
- `runtime-version` and `python-version` are both optional, and can have their values specified in the `config.yaml` as well. By default CMLE uses Python2.7, but can be set to use Python3.5. Runtime version is what version of CMLE to use. At the time of this writing 1.12 is the most up-to-date version. Different runtime versions have different versions of default packages installed on the VMs used by CMLE. Check out the documentation **LINK TO THE DOCUMENTATION**

You can monitor the job status from the console.

Distributed training and hyper-parameter tuning specifications are set in the `config.yaml` file, as per the API **LINK TO THE APIs**. The only difference between submitting a regular training job, a distributed training job, or a hyper-parameter tuning job is what is in the config file.

**Note:** It is *strongly* recommended that you test the job locally before submitting it to CMLE and incurring a cost. This is done by running
```
gcloud ml-engine local train
```
with the parameters as above, save for the job name, staging bucket, region, runtime and python versions.

**OPEN QUESTIONS FOR MYSELF:** 
1. How much can you deviate from the design pattern above?
2. What's the deal with job-dir as an argparse argument?
3. How does local training know what Python version to use? Is it just whatever is on the `PATH`?

## 4. Models and versions

## 5. Online and batch predictions